# Artificial Source Objects

To generate synthetic images, **ArtPop** "observes" [Source](https://artpop.readthedocs.io/en/latest/api/artpop.source.Source.html) objects. In essence, ``Source`` objects are containers that hold the ``xy`` pixel positions and stellar ``mags`` of the artificial stellar population.  As we showed in the [Building Stellar Populations](https://artpop.readthedocs.io/en/latest/tutorials/pops.html) and [Sampling Spatial Distributions](https://artpop.readthedocs.io/en/latest/tutorials/spatial.html) tutorials, **ArtPop** has the capability to generate these parameters, but you can generate them in _any_ way you want (i.e., independently from **ArtPop**). You just need to make sure to create ``xy`` and ``mags`` in the correct format to initialize the [Source](https://artpop.readthedocs.io/en/latest/api/artpop.source.Source.html).

**Note:** To generate MIST synthetic photometry using **ArtPop**, [MIST isochrone grids](https://artpop.readthedocs.io/en/latest/getting_started/mist.html) are required. The first time you use a MIST grid, **ArtPop** will download it and save it to your `MIST_PATH`. If this environmental variable is not set, the grid(s) will be saved in `~/.artpop/mist`.

### Install ArtPop and its dependencies

In [ ]:
!pip install artpop

In [ ]:
# Third-party imports 
import numpy as np
import matplotlib.pyplot as plt
from astropy import units as u   

# Project import
import artpop

# artpop's matplotlib style
plt.style.use(artpop.mpl_style)

# use this random state for reproducibility
rng = np.random.RandomState(9)

## Building a Source Object

In this first example, we will show you how to build a ``Source`` object from scratch. We'll assume a uniform distribution of stars at fixed surface brightness. We start by creating a simple stellar population (SSP), which we will use to calculate the mean stellar magnitude of the population:

In [ ]:
# Here we use total_mass instead of num_stars, 
# one of which is required. We use a relatively 
# large mass to ensure we fully sample the imf.
ssp = artpop.MISTSSP(
    log_age = 9,          # log of age in years
    feh = -1,             # metallicity [Fe/H]
    phot_system = 'LSST', # photometric system(s)
    total_mass = 1e6,     # total mass (in solar masses)
    imf = 'kroupa',       # default imf
    random_state = rng,   # random state for reproducibility
)

Next we use the [constant_sb_stars_per_pix](https://artpop.readthedocs.io/en/latest/api/artpop.stars.constant_sb_stars_per_pix.html) function to calculate the number of stars this population would have for a given surface brightness and distance (let's put the population at 10 Mpc):

In [ ]:
distance = 10 * u.Mpc

num_stars_per_pix = artpop.constant_sb_stars_per_pix(
    sb = 24,                           # surface brightness
    mean_mag = ssp.mean_mag('LSST_i'), # mean stellar magnitude
    distance = distance,               # distance to system
    pixel_scale = 0.2                  # pixel scale in arcsec/pixel
)

print(f'number of stars per pixel = {num_stars_per_pix:.2f}')

Let's say we intend to create an artificial image that is 101 pixels on a side. Then we can calculate the number of pixels, and hence number of stars, in our image.

In [ ]:
xy_dim = (101, 101)
num_stars = int(np.multiply(*xy_dim) * num_stars_per_pix)
print(f'number of stars = {num_stars:.0e}')

Putting the pieces together, we sample ``num_stars`` stellar magnitudes and positions to build our ``Source`` object:

In [ ]:
# build SSP with num_stars stars
ssp = artpop.MISTSSP(
    log_age = 9,           # log of age in years
    feh = -1,              # metallicity [Fe/H]
    phot_system = 'LSST',  # photometric system(s)
    num_stars = num_stars, # number of stars
    imf = 'kroupa',        # default imf
    distance = distance,   # distance to system
    random_state = rng,    # random state for reproducibility
)

# sample num_stars positions in a uniform grid
xy = np.vstack([rng.uniform(0, xy_dim[0], num_stars), 
                rng.uniform(0, xy_dim[1], num_stars)]).T

# create the artificial source
src = artpop.Source(xy, ssp.mag_table, xy_dim)

Note ``ssp`` has an attribute called ``mag_table``, which is an ``astropy`` table with stellar magnitudes in the given photometric system(s), which is passed as an argument to ``Source`` and stored as the ``mags`` attribute:

In [ ]:
# here are the first 10 rows of the table
# each row corresponds to a single star
src.mags[:10]

The ``src`` object contains all the information we need to simulate an observation of the artificial source, which is described in the [Making Artificial Images](https://artpop.readthedocs.io/en/latest/tutorials/artimages.html) tutorial. 


Here's a look at the positions and $i$-band magnitudes of a random subset of the stars in ``src``:

In [ ]:
idx = rng.choice(np.arange(src.num_stars), 5000, replace=False)
plt.scatter(src.x[idx], src.y[idx], s=4, 
            c=src.mags['LSST_i'][idx], cmap='magma')

cbar = plt.colorbar()
cbar.ax.set_ylabel('$m_i$')
plt.xlabel('$x$ [pixel]')
plt.ylabel('$y$ [pixel]')
plt.title(f'SSP at {distance:.0f}')
plt.minorticks_on();

As a sanity check, we can calculate the surface brightness of the population:

In [ ]:
total_flux = (10**(-0.4 * src.mags['LSST_i'])).sum()
area = (101 * 0.2)**2
sb = -2.5 * np.log10(total_flux / area)
print(f'SB = {sb:.2f} mag / arcsec^2')

## Helper SSP Source Objects

For convenience, **ArtPop** provides objects for generating SSP + spatial distribution combinations. Here we generate an SSP with a uniform spatial distribution, similar to above, using the [MISTUniformSSP](https://artpop.readthedocs.io/en/latest/api/artpop.source.MISTUniformSSP.html) helper SSP class:

In [ ]:
# let's put the population at 5 Mpc
distance = 5 * u.Mpc

# create ssp distributed uniformly in space
src_uniform = artpop.MISTUniformSSP(
    log_age = 9,          # log of age in years
    feh = -1,             # metallicity [Fe/H]
    phot_system = 'LSST', # photometric system(s) 
    distance = distance,  # distance to system 
    xy_dim = 101,         # image dimension (101, 101)
    pixel_scale = 0.2,    # pixel scale in arcsec / pixel
    sb = 24,              # surface brightness (SB)
    sb_band='LSST_i',     # bandpass to calculate the SB
    random_state = rng,   # random state for reproducibility    
)

# plot positions with symbols colored by i-band mags
idx = rng.choice(np.arange(src_uniform.num_stars), 5000, replace=False)
plt.scatter(src_uniform.x[idx], src_uniform.y[idx], s=4, 
            c=src_uniform.mags['LSST_i'][idx], cmap='magma')

cbar = plt.colorbar()
cbar.ax.set_ylabel('$m_i$')
plt.xlabel('$x$ [pixel]')
plt.ylabel('$y$ [pixel]')
plt.title(f'SSP at {distance:.0f}')
plt.minorticks_on();

## Composite Sources

Similar to [creating composite stellar populations](https://artpop.readthedocs.io/en/latest/tutorials/pops.html#Composite-Stellar-Populations), composite ``Source`` objects are created intuitively using the ``+`` operator. Here we'll use [MISTPlummerSSP](https://artpop.readthedocs.io/en/latest/api/artpop.source.MISTPlummerSSP.html) to create an SSP with a Plummer spatial distribution and add it to the uniformly distributed population we created above:

In [ ]:
# create ssp distributed uniformly in space
src_plummer = artpop.MISTPlummerSSP(
    log_age = 10.1,           # log of age in years
    feh = -1.5,               # metallicity [Fe/H]
    scale_radius = 20 * u.pc, # effective radius
    num_stars = 5e5,          # number of stars
    phot_system = 'LSST',     # photometric system(s) 
    distance = distance,      # distance to system 
    xy_dim = 101,             # image dimension (101, 101)
    pixel_scale = 0.2,    # pixel scale in arcsec / pixel
    random_state = rng,       # random state for reproducibility
)

# add sources together
composite_src = src_uniform + src_plummer

# plot positions with symbols colored by i-band mags
idx = rng.choice(np.arange(composite_src.num_stars), int(1e4), replace=False)
plt.scatter(composite_src.x[idx], composite_src.y[idx], s=2, 
            c=composite_src.mags['LSST_i'][idx], cmap='magma')

cbar = plt.colorbar()
cbar.ax.set_ylabel('$m_i$')
plt.xlabel('$x$ [pixel]')
plt.ylabel('$y$ [pixel]')
plt.title(f'SSP at {distance:.0f}')
plt.minorticks_on();